In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Скачиваем нужные данные:

In [2]:
tweets = pd.read_csv('https://raw.githubusercontent.com/evlko/CS-224W/main/Data/Twitter/tweets_covid19_GL.csv')

# Preprocessing 

In [3]:
import nltk
import re
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('movie_reviews', quiet=True)

True

Совершенно стандратный препроцессинг для текста 

In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [5]:
def remove_words_with_sym(text, symbol='#'):
  return re.sub(r'{}[^\s]*'.format(symbol), '', text)

assert remove_words_with_sym('Hello, it\'s a me, Mario!, #Top') == 'Hello, it\'s a me, Mario!, '
assert remove_words_with_sym('Hello, it\'s a me, @Mario!', '@') == 'Hello, it\'s a me, '

In [6]:
def preprocess(text, special_symbols=['#', '@', 'http', 'pic twitter c']):
    for special_symbol in special_symbols:
      text = remove_words_with_sym(text, special_symbol)
    text = re.sub(r'[^a-zA-Z\s]', '', text).replace('_', '')
    text = text.lower()
    text = [lemmatizer.lemmatize(word) for word in word_tokenize(text) if lemmatizer.lemmatize(word) not in stop_words] 
    text = ' '.join(text)

    return text

assert preprocess('Hello, it\'s a me, Mario!') == 'hello mario'

Обрабатываем необходимые данные:

In [7]:
tweets['text'] = tweets['text'].apply(preprocess)

Берем случайную подвыборку

In [8]:
tweets = tweets.sample(n=10000)

# GPT-2

In [ ]:
!pip install datasets accelerate transformers

In [10]:
import torch, os, re as json
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig
from datasets import Dataset

Меняем среду выполнения с CPU на GPU

In [11]:
dev = 'cuda:0' if torch.cuda.is_available() else 'cpu' 
device = torch.device(dev)  
device

device(type='cuda', index=0)

Создаем экземпляр модели из зараннее обученной

In [ ]:
base_model = GPT2LMHeadModel.from_pretrained('gpt2')

base_model.num_parameters

Создаем экземпляр токенайзера из зараннее обученного

In [13]:
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

## Базовый пайплайн генерации

Выберем случайный текст -> превартим его в набор токенов -> заэнкодим токены через токенайзер

In [14]:
tweet = tweets.sample(n=1)['text'].to_list()[0]
base_tokenizer.tokenize(tweet)
text_ids = base_tokenizer.encode(tweet, return_tensors = 'pt')
text_ids

tensor([[29214,  1672,   302,  7329,  9343,   300, 11690,  1628,  2099, 41477,
          8639,  5968,  6794]])

Теперь сгенерируем новый текст -> задекодим его через токенайзер -> вывелем оригинал и сгенерированное

In [15]:
generated_text_samples = base_model.generate(
    text_ids
)

print('Original text: %s' % tweet)

generated_tweet = ''

for i, beam in enumerate(generated_text_samples):
    generated_tweet += ' ' + base_tokenizer.decode(beam, skip_special_tokens=True)

print('Generated text: %s' % generated_tweet)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original text: another example reagan enabled lincoln project type republican contributed hellscape
Generated text:  another example reagan enabled lincoln project type republican contributed hellscape to the project.

The


Всего есть 4 вида "метода" генерации:
1. Greedy. Берем следующее слово с макс. вероятностью;
2. Beam. Считаем цепочку с макс. вероятностью;
3. Sampling. Следующее слово выбирается случайным образом на основе распределения вероятностей, обусловленного предыдущими словами.

Создадим универсальную функцию для генерации текста

In [16]:
def generate_text(model, tokenizer, input_text, device):
  text_ids = tokenizer.encode(input_text, return_tensors = 'pt')
  text_ids = text_ids.to(device)
  model = model.to(device)

  generated_text_samples = model.generate(
    text_ids, 
    no_repeat_ngram_size=2,
    repetition_penalty= 1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=False
  )
  
  gen_text = ''
  for t in generated_text_samples:
    gen_text = tokenizer.decode(t, skip_special_tokens=True)

  return gen_text

## Fine-tunning

В первую очередь дополним токенайзер. Создадим специальным токены, которые будут отделять одну последоватеьность от другой. Далее создадим новые экземпляр модели и токенайзера.

In [17]:
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}

base_tokenizer.add_special_tokens(special_tokens_dict)

config = AutoConfig.from_pretrained('gpt2', 
                                    bos_token_id=base_tokenizer.bos_token_id,
                                    eos_token_id=base_tokenizer.eos_token_id,
                                    pad_token_id=base_tokenizer.pad_token_id,
                                    output_hidden_states=False)

base_model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)

base_model.resize_token_embeddings(len(base_tokenizer))

Embedding(50259, 768)

Дополним сам текст этими самыми специальными токенам и разделим его на обучающую и тестовую выборку.

In [18]:
tweets['text'] = bos + ' ' + tweets['text'] + ' ' + eos

tweets_train, tweets_test = train_test_split(tweets, train_size=0.8)

Перейдем от `pd.DataFrame` к `Dataset`

In [19]:
train_dataset = Dataset.from_pandas(tweets_train[['text']])
test_dataset = Dataset.from_pandas(tweets_test[['text']])

Сделаем токенизацию

In [20]:
def tokenize_function(df):
  return base_tokenizer(df['text'], padding=True)

tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=['text'],
)
tokenized_val_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=['text'],
)

Map (num_proc=5):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=5):   0%|          | 0/2000 [00:00<?, ? examples/s]

Определим параметры модели, данных и обучения. Запустим обучение

In [21]:
model_path = './model_tweets'

training_args = TrainingArguments(
  output_dir=model_path,
  num_train_epochs=6,    
  per_device_train_batch_size=32,
  per_device_eval_batch_size=16,
  warmup_steps=200,    
  weight_decay=0.01,
  logging_dir=model_path, 
  prediction_loss_only=True,
  save_steps=10000 
)

data_collator = DataCollatorForLanguageModeling(
  tokenizer=base_tokenizer,
  mlm=False
)

trainer = Trainer(
  model=base_model,                      
  args=training_args,      
  data_collator=data_collator,
  train_dataset=tokenized_train_dataset,  
  eval_dataset=tokenized_val_dataset
)

trainer.train()

Step,Training Loss
500,10.820700
1000,5.811100
1500,5.489100


TrainOutput(global_step=1500, training_loss=7.373604654947917, metrics={'train_runtime': 464.0542, 'train_samples_per_second': 103.436, 'train_steps_per_second': 3.232, 'total_flos': 828622356480000.0, 'train_loss': 7.373604654947917, 'epoch': 6.0})

Сохраним модель

In [22]:
trainer.save_model()
base_tokenizer.save_pretrained(model_path)

('./model_tweets/tokenizer_config.json',
 './model_tweets/special_tokens_map.json',
 './model_tweets/vocab.json',
 './model_tweets/merges.txt',
 './model_tweets/added_tokens.json')

Оцени модель

In [23]:
trainer.evaluate()

{'eval_loss': 6.1307244300842285,
 'eval_runtime': 4.8495,
 'eval_samples_per_second': 412.411,
 'eval_steps_per_second': 25.776,
 'epoch': 6.0}

Загрузим модель

In [24]:
tweets_model = GPT2LMHeadModel.from_pretrained(model_path)
tweets_tokenizer = GPT2Tokenizer.from_pretrained(model_path)

Сгенерируем новый твит. Причём стоит обратить внимание на параметры генерации. Если оставить просто `Greedy`, то каждый раз мы будем получать один и тот же твит

In [29]:
input_text = 'covid'

generated_tweet = generate_text(tweets_model, tweets_tokenizer, input_text, device)

generated_tweet

'covid case reported highest singleday spike new death reported h'